In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import networkx as nx
import netwulf as nw
import numpy as np


In [7]:
amount_of_nodes = 14196
amount_of_edges = 54304
probability = amount_of_edges/((amount_of_nodes*(amount_of_nodes-1))/2)
print(probability)
k = probability*(amount_of_nodes-1.0)
print(k)

def generate_random_network(node_count, p):
    G = nx.Graph()
    
    G.add_nodes_from(range(node_count))
    for i in range(node_count):
        for j in range(i + 1, node_count):  
            if np.random.uniform(0, 1) < p:
                G.add_edge(i, j)
    
    return G

#random_network = generate_random_network(amount_of_nodes,probability)
#nx.draw(random_network)
#nw.interactive.visualize (random_network)
RG = nx.gnp_random_graph(amount_of_nodes, probability, seed=1000, directed=False)
nw.interactive.visualize(RG)


0.0005389648330011947
7.650605804451958


(None, None)